# Laboratorio 2

- Brando Reyes
- Juan Pablo Solis

## Task 1 - Preguntas Teoricas

1. ¿Por qué el modelo de Naive Bayes se le considera "naive"?
    - Este modelo se considera "Naive" debido a que asume que todas las características son independientes entre sí, lo cual rara vez es cierto en problemas del mundo real
2. Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda ¿cómo funciona el truco del Kernel para este modelo?
3. Investigue sobre Random Forest y responda:
    -  ¿Qué tipo de ensemble learning es este modelo?
        - Random Forest es un modelo de bagging el cual combina múltiples modelos que se entrenan de manera independiente utilizando diferentes subconjuntos de datos, y luego promedia o vota para obtener la predicción final.
    - ¿Cuál es la idea general detrás de Random Forest?
        - La idea general de Random Forest es combinar múltiples árboles de decisión para mejorar la precisión y reducir el riesgo de sobreajuste. 
    - ¿Por qué se busca baja correlación entre los árboles de Random Forest?
        - La baja correlación entre los árboles es clave para el éxito de Random Forest porque aumenta su capacidad de generalización. Si los árboles son altamente correlacionados, cometerán los mismos errores, lo que limitaría la mejora obtenida al combinarlos

# Task 2 Lectura y limpieza de archivo

In [1]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score

In [2]:
# Leer el archivo y cargarlo en un DataFrame
def cargar_datos(ruta_archivo):
    mensajes = []
    etiquetas = []
    
    with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
        for linea in archivo:
            if "\t" in linea:
                etiqueta, mensaje = linea.strip().split("\t", 1)
                etiquetas.append(etiqueta.lower())
                mensajes.append(mensaje.lower())  # Convertir a minúsculas para uniformidad
    
    return pd.DataFrame({'etiqueta': etiquetas, 'mensaje': mensajes})


# Procesar los datos
ruta_archivo = 'entrenamiento.txt'  # Cambiar por la ruta correcta del archivo
df = cargar_datos(ruta_archivo)

print(df.head())

# Verificar la distribución de clases
print("\nDistribución de clases:")
print(df["etiqueta"].value_counts())


  etiqueta                                            mensaje
0      ham  go until jurong point, crazy.. available only ...
1      ham                      ok lar... joking wif u oni...
2     spam  free entry in 2 a wkly comp to win fa cup fina...
3      ham  u dun say so early hor... u c already then say...
4      ham  nah i don't think he goes to usf, he lives aro...

Distribución de clases:
etiqueta
ham     4818
spam     747
Name: count, dtype: int64


In [3]:
df.info

<bound method DataFrame.info of      etiqueta                                            mensaje
0         ham  go until jurong point, crazy.. available only ...
1         ham                      ok lar... joking wif u oni...
2        spam  free entry in 2 a wkly comp to win fa cup fina...
3         ham  u dun say so early hor... u c already then say...
4         ham  nah i don't think he goes to usf, he lives aro...
...       ...                                                ...
5560     spam  this is the 2nd time we have tried 2 contact u...
5561      ham               will ü b going to esplanade fr home?
5562      ham  pity, * was in mood for that. so...any other s...
5563      ham  the guy did some bitching but i acted like i'd...
5564      ham                         rofl. its true to its name

[5565 rows x 2 columns]>

### Limpiar el dataset

In [4]:

# Función para limpiar el texto
def limpiar_mensaje(mensaje):

    # Eliminar caracteres especiales excepto letras, números y espacios
    mensaje = re.sub(r"[^a-zA-Z0-9\s]", '', mensaje)
    # Quitar espacios adicionales
    mensaje = re.sub(r'\s+', ' ', mensaje).strip()
    # Convertir a minúsculas
    return mensaje.lower()

#Aplicar la limpieza al DataFrame
df['mensaje'] = df['mensaje'].apply(limpiar_mensaje)

# Verificar los resultados
print(df.head())


  etiqueta                                            mensaje
0      ham  go until jurong point crazy available only in ...
1      ham                            ok lar joking wif u oni
2     spam  free entry in 2 a wkly comp to win fa cup fina...
3      ham        u dun say so early hor u c already then say
4      ham  nah i dont think he goes to usf he lives aroun...


## Dividir el dataset

In [5]:

# Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['etiqueta'])

# Verificar las formas de los conjuntos
print("Tamaño del conjunto de entrenamiento:", train_df.shape)
print("Tamaño del conjunto de prueba:", test_df.shape)

# Opcional: Verificar la distribución de clases en ambos conjuntos
print("\nDistribución en el conjunto de entrenamiento:")
print(train_df['etiqueta'].value_counts(normalize=True))

print("\nDistribución en el conjunto de prueba:")
print(test_df['etiqueta'].value_counts(normalize=True))


Tamaño del conjunto de entrenamiento: (4452, 2)
Tamaño del conjunto de prueba: (1113, 2)

Distribución en el conjunto de entrenamiento:
etiqueta
ham     0.865678
spam    0.134322
Name: proportion, dtype: float64

Distribución en el conjunto de prueba:
etiqueta
ham     0.866128
spam    0.133872
Name: proportion, dtype: float64


## Construccion del modelo

In [6]:
# Función para entrenar el modelo basado en Bayes con Laplace Smoothing 
def entrenar_bayes(mensajes, etiquetas, alpha=1.0):
    # Inicializar contadores
    total_mensajes = len(mensajes)
    total_por_categoria = {}
    palabras_por_categoria = {}
    total_palabras_por_categoria = {}
    vocabulario = set()
    
    # Procesar cada mensaje
    for i in range(len(mensajes)):
        etiqueta = etiquetas[i]
        mensaje = limpiar_mensaje(mensajes[i])  # Llamar a la función de limpieza
        palabras = mensaje.split()
        
        # Inicializar estructuras para la categoría si no existen
        if etiqueta not in total_por_categoria:
            total_por_categoria[etiqueta] = 0
            palabras_por_categoria[etiqueta] = {}
            total_palabras_por_categoria[etiqueta] = 0
        
        # Actualizar contadores
        total_por_categoria[etiqueta] += 1
        total_palabras_por_categoria[etiqueta] += len(palabras)
        
        for palabra in palabras:
            vocabulario.add(palabra)
            if palabra not in palabras_por_categoria[etiqueta]:
                palabras_por_categoria[etiqueta][palabra] = 0
            palabras_por_categoria[etiqueta][palabra] += 1
    
    # Calcular probabilidades con Laplace Smoothing
    tamaño_vocabulario = len(vocabulario)
    probabilidad_categoria = {cat: total_por_categoria[cat] / total_mensajes for cat in total_por_categoria}
    probabilidad_palabra_dado_categoria = {}
    
    for categoria in palabras_por_categoria:
        probabilidad_palabra_dado_categoria[categoria] = {}
        for palabra in vocabulario:
            conteo_palabra = palabras_por_categoria[categoria].get(palabra, 0)
            probabilidad_palabra_dado_categoria[categoria][palabra] = (conteo_palabra + alpha) / (total_palabras_por_categoria[categoria] + alpha * tamaño_vocabulario)
    
    return probabilidad_categoria, probabilidad_palabra_dado_categoria, tamaño_vocabulario, vocabulario

# Función para predecir la categoría de un mensaje
def predecir_bayes_flexible(mensaje, prob_categoria, prob_palabra_categoria, tamaño_vocabulario, alpha=1.0):
    mensaje = limpiar_mensaje(mensaje)  # Llamar a la función de limpieza antes de predecir
    palabras = mensaje.split()
    probabilidades = {}
    
    for categoria in prob_categoria:
        # Iniciar con la probabilidad de la categoría
        probabilidad = prob_categoria[categoria]
        
        # Multiplicar las probabilidades de las palabras
        for palabra in palabras:
            if palabra in prob_palabra_categoria[categoria]:
                probabilidad *= prob_palabra_categoria[categoria][palabra]
            else:
                # Manejar palabras desconocidas con Laplace Smoothing
                probabilidad *= alpha / (tamaño_vocabulario + alpha)
        
        probabilidades[categoria] = probabilidad
    
    # Devolver la categoría con mayor probabilidad
    return max(probabilidades, key=probabilidades.get)

# Entrenar el modelo usando el conjunto de entrenamiento
X_train = train_df['mensaje'].tolist()  # Convertir a lista si es necesario
y_train = train_df['etiqueta'].tolist()
prob_categoria, prob_palabra_categoria, tamaño_vocabulario, vocabulario = entrenar_bayes(X_train, y_train, alpha=0.5)

# Imprimir estadísticas del modelo entrenado
print("Tamaño del vocabulario:", tamaño_vocabulario)

# Probar el modelo con el conjunto de prueba
X_test = test_df['mensaje'].tolist()
y_test = test_df['etiqueta'].tolist()

correctos = 0
for i in range(len(X_test)):
    mensaje = X_test[i]
    etiqueta_real = y_test[i]
    etiqueta_predicha = predecir_bayes_flexible(mensaje, prob_categoria, prob_palabra_categoria, tamaño_vocabulario, alpha=0.5)
    
    if etiqueta_predicha == etiqueta_real:
        correctos += 1

# Calcular la precisión
precision = correctos / len(X_test)
print(f"Precisión del modelo: {precision:.2f}")


Tamaño del vocabulario: 8353
Precisión del modelo: 0.98


### Prueba Con metrica Recall

- Creemos que recall es una metrica adecuada debido a que permite minimizar los falsos negativos. En este caso al tener spam como una clase minoritaria  queremos asegurarnos de que el modelo pueda identificar correctamente la mayor cantidad de mensajes spam

In [7]:
# Función para calcular el Recall por clase
def calcular_recall(y_real, y_pred, clase_objetivo):
    verdaderos_positivos = 0
    falsos_negativos = 0

    for i in range(len(y_real)):
        if y_real[i] == clase_objetivo:
            if y_pred[i] == clase_objetivo:
                verdaderos_positivos += 1
            else:
                falsos_negativos += 1
    
    # Recall = Verdaderos Positivos / (Verdaderos Positivos + Falsos Negativos)
    if (verdaderos_positivos + falsos_negativos) == 0:
        return 0  # Evitar división por cero
    return verdaderos_positivos / (verdaderos_positivos + falsos_negativos)

# Evaluar el modelo en el conjunto de entrenamiento
y_train_pred = [predecir_bayes_flexible(mensaje, prob_categoria, prob_palabra_categoria, tamaño_vocabulario, alpha=0.5) for mensaje in X_train]
recall_train_spam = calcular_recall(y_train, y_train_pred, clase_objetivo="spam")
recall_train_ham = calcular_recall(y_train, y_train_pred, clase_objetivo="ham")
recall_train_promedio = (recall_train_spam + recall_train_ham) / 2

# Evaluar el modelo en el conjunto de prueba
y_test_pred = [predecir_bayes_flexible(mensaje, prob_categoria, prob_palabra_categoria, tamaño_vocabulario, alpha=0.5) for mensaje in X_test]
recall_test_spam = calcular_recall(y_test, y_test_pred, clase_objetivo="spam")
recall_test_ham = calcular_recall(y_test, y_test_pred, clase_objetivo="ham")
recall_test_promedio = (recall_test_spam + recall_test_ham) / 2

# Presentar métricas de desempeño con el formato requerido
print("=== Entrenamiento manual  ===\n")

print("=== Evaluación en conjunto de entrenamiento ===\n")
print("Métricas de evaluación:")
print(f"- Recall para spam: {recall_train_spam:.3f}")
print(f"- Recall para ham: {recall_train_ham:.3f}")
print(f"- Recall promedio: {recall_train_promedio:.3f}\n")

print("=== Evaluación en conjunto de prueba ===\n")
print("Métricas de evaluación:")
print(f"- Recall para spam: {recall_test_spam:.3f}")
print(f"- Recall para ham: {recall_test_ham:.3f}")
print(f"- Recall promedio: {recall_test_promedio:.3f}")


=== Entrenamiento manual  ===

=== Evaluación en conjunto de entrenamiento ===

Métricas de evaluación:
- Recall para spam: 0.975
- Recall para ham: 0.997
- Recall promedio: 0.986

=== Evaluación en conjunto de prueba ===

Métricas de evaluación:
- Recall para spam: 0.886
- Recall para ham: 0.995
- Recall promedio: 0.940


##  2.3 Mensajes futuros

In [8]:
# Bucle para entrada de mensajes en consola hasta que el usuario escriba "z"
print("\n=== Clasificación interactiva de mensajes ===")
print("Ingrese un mensaje para clasificarlo. Escriba 'z' para salir.\n")

while True:
    mensaje_usuario = input("Mensaje: ").strip()
    
    if mensaje_usuario.lower() == "z":
        print("Saliendo del clasificador...")
        break  # Detener la ejecución cuando el usuario escriba 'z'
    
    palabras = mensaje_usuario.split()
    probabilidades = {}

    # Calcular las probabilidades de spam y ham
    for categoria in prob_categoria:
        probabilidad = prob_categoria[categoria]  # P(Categoria)
        
        for palabra in palabras:
            if palabra in prob_palabra_categoria[categoria]:
                probabilidad *= prob_palabra_categoria[categoria][palabra]
            else:
                # Manejar palabras desconocidas con Laplace Smoothing
                probabilidad *= 0.5 / (tamaño_vocabulario + 0.5)
        
        probabilidades[categoria] = probabilidad

    # Normalizar las probabilidades para obtener una distribución entre 0 y 1
    suma_probabilidades = sum(probabilidades.values())
    if suma_probabilidades > 0:
        for categoria in probabilidades:
            probabilidades[categoria] /= suma_probabilidades

    # Determinar la categoría con mayor probabilidad
    categoria_predicha = max(probabilidades, key=probabilidades.get)

    # Mostrar resultados en consola
    print("\n--- Resultados de clasificación ---")
    print(f"Mensaje ingresado: \"{mensaje_usuario}\"")
    print(f"Probabilidad de SPAM: {probabilidades['spam']:.3f}")
    print(f"Probabilidad de HAM: {probabilidades['ham']:.3f}")
    print(f"Clasificación: {categoria_predicha.upper()}\n")



=== Clasificación interactiva de mensajes ===
Ingrese un mensaje para clasificarlo. Escriba 'z' para salir.


--- Resultados de clasificación ---
Mensaje ingresado: "red car"
Probabilidad de SPAM: 0.014
Probabilidad de HAM: 0.986
Clasificación: HAM


--- Resultados de clasificación ---
Mensaje ingresado: "membership"
Probabilidad de SPAM: 0.327
Probabilidad de HAM: 0.673
Clasificación: HAM

Saliendo del clasificador...


## 2.4 Comparacion de librerias

In [9]:


# Convertir los mensajes en formato numérico usando Bag of Words
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)  # Ajustar y transformar training
X_test_vectorized = vectorizer.transform(X_test)  # Solo transformar testing

# Entrenar el modelo Naïve Bayes con sklearn
modelo_sklearn = MultinomialNB(alpha=0.5)  # Alpha para Laplace Smoothing
modelo_sklearn.fit(X_train_vectorized, y_train)

# Predicciones en training y testing
y_train_pred_sklearn = modelo_sklearn.predict(X_train_vectorized)
y_test_pred_sklearn = modelo_sklearn.predict(X_test_vectorized)

# Calcular Recall en training y testing
recall_train_spam_sklearn = recall_score(y_train, y_train_pred_sklearn, pos_label="spam")
recall_train_ham_sklearn = recall_score(y_train, y_train_pred_sklearn, pos_label="ham")
recall_train_promedio_sklearn = (recall_train_spam_sklearn + recall_train_ham_sklearn) / 2

recall_test_spam_sklearn = recall_score(y_test, y_test_pred_sklearn, pos_label="spam")
recall_test_ham_sklearn = recall_score(y_test, y_test_pred_sklearn, pos_label="ham")
recall_test_promedio_sklearn = (recall_test_spam_sklearn + recall_test_ham_sklearn) / 2

# Presentar métricas de desempeño con el mismo formato que en la implementación manual
print("=== Evaluación con Naïve Bayes de sklearn ===\n")

print("=== Evaluación en conjunto de entrenamiento ===\n")
print("Métricas de evaluación:")
print(f"- Recall para spam: {recall_train_spam_sklearn:.3f}")
print(f"- Recall para ham: {recall_train_ham_sklearn:.3f}")
print(f"- Recall promedio: {recall_train_promedio_sklearn:.3f}\n")

print("=== Evaluación en conjunto de prueba ===\n")
print("Métricas de evaluación:")
print(f"- Recall para spam: {recall_test_spam_sklearn:.3f}")
print(f"- Recall para ham: {recall_test_ham_sklearn:.3f}")
print(f"- Recall promedio: {recall_test_promedio_sklearn:.3f}")


=== Evaluación con Naïve Bayes de sklearn ===

=== Evaluación en conjunto de entrenamiento ===

Métricas de evaluación:
- Recall para spam: 0.977
- Recall para ham: 0.997
- Recall promedio: 0.987

=== Evaluación en conjunto de prueba ===

Métricas de evaluación:
- Recall para spam: 0.893
- Recall para ham: 0.994
- Recall promedio: 0.943


### ¿Cuál implementación lo hizo mejor? ¿Su implementación o la de la librería?
    - Ambas pruebas de recall obtuvieron valores similares
        - La implementacion con Sklearn tiene mejor Recall para Spam (+0.007), lo que significa que detecta más mensajes spam correctamente.
        - Manual tiene mejor Recall para Ham (+0.001), aunque la diferencia es mínima.
        - Sklearn tiene mejor Recall Promedio (+0.003), lo que indica un mejor balance en ambas clases.
### ¿Por qué cree que se debe esta diferencia?
    - La principal diferencia es que sklearn maneja mejor los cálculos numéricos usando logaritmos en lugar de multiplicaciones, evitando errores de precisión y subdesbordamiento. Además, CountVectorizer preprocesa mejor el texto, eliminando ruido y mejorando la representación del mensaje

# Task 3 - Clasificación de Partidas de League of Legends

In [10]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['etiqueta'])

# Resumen de las divisiones
train_summary = train_df['etiqueta'].value_counts(normalize=True) * 100
test_summary = test_df['etiqueta'].value_counts(normalize=True) * 100

# Mostrar los resúmenes de las divisiones
train_summary, test_summary


(etiqueta
 ham     86.567835
 spam    13.432165
 Name: proportion, dtype: float64,
 etiqueta
 ham     86.612758
 spam    13.387242
 Name: proportion, dtype: float64)

In [11]:
# Reinicio del flujo de trabajo desde la carga del dataset
# Asegurarse de que el dataset df esté definido antes de continuar

# Simulación de una estructura básica para el dataframe df (puedes reemplazarlo con el real)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Definir un dataset de ejemplo para simular el proceso (sustituir con el dataset real)
data = {
    'etiqueta': ['ham', 'spam', 'ham', 'spam', 'ham'],
    'caracteristica_1': [10, 20, 10, 30, 10],
    'caracteristica_2': [1.5, 2.5, 1.5, 3.5, 1.5]
}
#df = pd.DataFrame(data)
df = df
# Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['etiqueta'])

# Crear una copia para preprocesamiento
train_processed = train_df.copy()
test_processed = test_df.copy()

# Paso 1: Encoding de la etiqueta (ham/spam -> 0/1)
label_encoder = LabelEncoder()
train_processed['etiqueta'] = label_encoder.fit_transform(train_processed['etiqueta'])
test_processed['etiqueta'] = label_encoder.transform(test_processed['etiqueta'])

# Paso 2: Escalar características numéricas
numeric_cols = train_processed.select_dtypes(include=['float64', 'int']).columns

scaler = StandardScaler()
train_processed[numeric_cols] = scaler.fit_transform(train_processed[numeric_cols])
test_processed[numeric_cols] = scaler.transform(test_processed[numeric_cols])

# Mostrar los resultados procesados
train_processed, test_processed


(      etiqueta                                            mensaje
 3223 -0.393908                   no plm i will come da on the way
 3289  2.538664  todays voda numbers ending 5226 are selected t...
 4399 -0.393908  awesome plan to get here any time after like l...
 5410 -0.393908             nope i just forgot will show next week
 1536 -0.393908                           midnight at the earliest
 ...        ...                                                ...
 4987 -0.393908  just looked it up and addie goes back monday s...
 60   -0.393908  your gonna have to pick up a 1 burger for your...
 3670  2.538664  promotion number 8714714 ur awarded a city bre...
 1710  2.538664  hard live 121 chat just 60pmin choose your gir...
 3777 -0.393908  let me know if you need anything else salad or...
 
 [4452 rows x 2 columns],
       etiqueta                                            mensaje
 3682 -0.393908                           what happen dear tell me
 3334 -0.393908                   

In [12]:
# Importar librerías necesarias
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Supongamos que 'df' ya está definido y limpio
# Dividir el dataset en entrenamiento (80%) y prueba (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['etiqueta'])

# Codificar etiquetas (ham/spam -> 0/1)
label_encoder = LabelEncoder()
train_df['etiqueta'] = label_encoder.fit_transform(train_df['etiqueta'])
test_df['etiqueta'] = label_encoder.transform(test_df['etiqueta'])

# Vectorizar los mensajes usando TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500)  # Máximo de 500 palabras más frecuentes
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['mensaje']).toarray()
X_test_tfidf = tfidf_vectorizer.transform(test_df['mensaje']).toarray()

# Etiquetas para entrenamiento y prueba
y_train = train_df['etiqueta']
y_test = test_df['etiqueta']

# Entrenar un modelo SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_tfidf, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = svm_model.predict(X_test_tfidf)

# Evaluar el modelo
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
conf_matrix = confusion_matrix(y_test, y_pred)

# Mostrar los resultados
print("Reporte de Clasificación:")
print(report)
print("\nMatriz de Confusión:")
print(conf_matrix)




Reporte de Clasificación:
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       964
        spam       0.97      0.87      0.92       149

    accuracy                           0.98      1113
   macro avg       0.98      0.93      0.95      1113
weighted avg       0.98      0.98      0.98      1113


Matriz de Confusión:
[[960   4]
 [ 19 130]]


En este caso, elegimos el **F1-score** como métrica de desempeño principal porque nos permite balancear precisión y recall, lo cual es crucial en problemas como la clasificación de spam, donde el dataset suele estar desbalanceado. Esta métrica armoniza la capacidad del modelo para identificar correctamente los mensajes spam (precisión) y para no dejar mensajes spam sin clasificar (recall). Además, ambos errores tienen implicaciones importantes: los falsos positivos afectan la experiencia del usuario al clasificar mensajes válidos como spam, mientras que los falsos negativos permiten que mensajes no deseados pasen desapercibidos. Con un F1-score promedio del 95%, el modelo demuestra un buen desempeño equilibrado en ambas clases, lo que lo hace adecuado para este tipo de problema.

# Task 3.2 - Support Vector Machines: Clasificación de Partidas de League of Legends

In [14]:
import pandas as pd

# Cargar el dataset proporcionado
dataset_path = "high_diamond_ranked_10min.csv"
df = pd.read_csv(dataset_path)

# Mostrar las primeras filas para verificar la estructura
df.head(), df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9879 entries, 0 to 9878
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   gameId                        9879 non-null   int64  
 1   blueWins                      9879 non-null   int64  
 2   blueWardsPlaced               9879 non-null   int64  
 3   blueWardsDestroyed            9879 non-null   int64  
 4   blueFirstBlood                9879 non-null   int64  
 5   blueKills                     9879 non-null   int64  
 6   blueDeaths                    9879 non-null   int64  
 7   blueAssists                   9879 non-null   int64  
 8   blueEliteMonsters             9879 non-null   int64  
 9   blueDragons                   9879 non-null   int64  
 10  blueHeralds                   9879 non-null   int64  
 11  blueTowersDestroyed           9879 non-null   int64  
 12  blueTotalGold                 9879 non-null   int64  
 13  blu

(       gameId  blueWins  blueWardsPlaced  blueWardsDestroyed  blueFirstBlood  \
 0  4519157822         0               28                   2               1   
 1  4523371949         0               12                   1               0   
 2  4521474530         0               15                   0               0   
 3  4524384067         0               43                   1               0   
 4  4436033771         0               75                   4               0   
 
    blueKills  blueDeaths  blueAssists  blueEliteMonsters  blueDragons  ...  \
 0          9           6           11                  0            0  ...   
 1          5           5            5                  0            0  ...   
 2          7          11            4                  1            1  ...   
 3          4           5            5                  1            0  ...   
 4          6           6            6                  0            0  ...   
 
    redTowersDestroyed  redTotalGold

In [15]:
# Seleccionar las columnas relevantes (dos para graficar y la variable objetivo)
columns_to_use = ['blueGoldPerMin', 'blueCSPerMin', 'blueWins']
df_subset = df[columns_to_use]

# Dividir el dataset en entrenamiento (80%), validación (10%) y prueba (10%)
train_data, temp_data = train_test_split(
    df_subset, test_size=0.2, random_state=42, stratify=df_subset['blueWins']
)
val_data, test_data = train_test_split(
    temp_data, test_size=0.5, random_state=42, stratify=temp_data['blueWins']
)

# Separar características y etiquetas
X_train = train_data[['blueGoldPerMin', 'blueCSPerMin']].values
y_train = train_data['blueWins'].values
X_val = val_data[['blueGoldPerMin', 'blueCSPerMin']].values
y_val = val_data['blueWins'].values
X_test = test_data[['blueGoldPerMin', 'blueCSPerMin']].values
y_test = test_data['blueWins'].values

# Verificar las divisiones
X_train.shape, X_val.shape, X_test.shape


((7903, 2), (988, 2), (988, 2))

In [17]:
import numpy as np

# Clase SVM básica
class SimpleSVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.learning_rate = learning_rate  # Tasa de aprendizaje
        self.lambda_param = lambda_param  # Parámetro de regularización
        self.n_iters = n_iters  # Número de iteraciones
        self.w = None  # Pesos
        self.b = None  # Sesgo

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)  # Convertir etiquetas 0 a -1
        self.w = np.zeros(n_features)  # Inicializar pesos con ceros
        self.b = 0  # Inicializar sesgo a 0

        # Gradiente descendente usando while
        iteration = 0
        while iteration < self.n_iters:
            idx = 0
            while idx < n_samples:
                # Condición para verificar si el ejemplo cumple con la regla
                condition = y_[idx] * (np.dot(X[idx], self.w) - self.b) >= 1
                if condition:
                    # Solo actualizar los pesos por regularización
                    self.w -= self.learning_rate * (2 * self.lambda_param * self.w)
                else:
                    # Actualizar pesos y sesgo
                    self.w -= self.learning_rate * (
                        2 * self.lambda_param * self.w - np.dot(X[idx], y_[idx])
                    )
                    self.b -= self.learning_rate * y_[idx]
                idx += 1  # Avanzar al siguiente ejemplo
            iteration += 1  # Incrementar la iteración

    def predict(self, X):
        # Calcular las predicciones
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

# Crear y entrenar el modelo SVM
svm = SimpleSVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)
svm.fit(X_train, y_train)

# Predicción en el conjunto de validación
y_val_pred = svm.predict(X_val)

# Evaluación del modelo
accuracy = np.mean(y_val == (y_val_pred > 0))

# Mostrar el resultado
print("Accuracy en validación (SVM desde cero con while):", accuracy)


Accuracy en validación (SVM desde cero con while): 0.49898785425101216


In [18]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Crear y entrenar el modelo SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Predicción y evaluación
y_val_pred = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)

print("Accuracy en validación (SVM con librerías):", accuracy)

# Reporte de clasificación y matriz de confusión
print("Reporte de Clasificación:")
print(classification_report(y_val, y_val_pred))
print("\nMatriz de Confusión:")
print(confusion_matrix(y_val, y_val_pred))


Accuracy en validación (SVM con librerías): 0.6933198380566802
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.69      0.70      0.69       495
           1       0.69      0.69      0.69       493

    accuracy                           0.69       988
   macro avg       0.69      0.69      0.69       988
weighted avg       0.69      0.69      0.69       988


Matriz de Confusión:
[[345 150]
 [153 340]]


La implementación de **SVM con librerías** fue significativamente mejor que la implementación desde cero. Utilizando Scikit-learn, se logró un **accuracy de 69.33%** en el conjunto de validación, mientras que la implementación desde cero alcanzó únicamente un 49.90%. Esto se debe a que Scikit-learn optimiza el entrenamiento mediante operaciones matriciales y algoritmos eficientes, lo que permite un mejor ajuste de las fronteras de decisión. Además, la versión con librerías ofrece herramientas para ajustar parámetros como el kernel y la regularización, lo que mejora el desempeño del modelo. En contraste, la implementación desde cero, aunque útil para entender los fundamentos del algoritmo, es menos eficiente y no logra capturar las relaciones complejas en los datos.

Por otro lado, el tiempo de ejecución fue notablemente menor en la implementación con librerías. La versión desde cero utiliza bucles `while`, lo que aumenta el tiempo de entrenamiento, especialmente con un dataset de tamaño considerable. A pesar de que la implementación manual fomenta el aprendizaje del funcionamiento interno del SVM, para problemas reales y más complejos, el uso de librerías es mucho más práctico y efectivo. En conclusión, Scikit-learn no solo ofrece mejores resultados en términos de precisión, sino que también permite un desarrollo más eficiente y ajustado a las necesidades del proyecto.

# Task 3.3 - Comparación


Task 3.3 - Comparación
¿Cómo difirieron los grupos creados por ambos modelos?
Los grupos creados por el modelo implementado desde cero y el modelo de librerías fueron bastante distintos en calidad. El modelo desde cero obtuvo un accuracy de 49.90%, lo que refleja que no logró ajustar correctamente las fronteras de decisión, provocando grupos poco definidos y clasificando de manera casi aleatoria. En cambio, el modelo con librerías alcanzó un accuracy de 69.33%, mostrando una clara mejora al identificar patrones en los datos y formando grupos más precisos. Esto es resultado de las optimizaciones matemáticas y de los ajustes de hiperparámetros que Scikit-learn realiza de forma eficiente.

¿Cuál de los modelos fue más rápido?
El modelo con librerías fue mucho más rápido en comparación con el modelo desde cero. Este último tardó 264 segundos, mientras que el modelo con librerías completó su entrenamiento y predicción en solo 8.7 segundos. Esto se debe a que Scikit-learn utiliza cálculos matriciales optimizados, mientras que la implementación desde cero dependió de ciclos while, lo que ralentiza el procesamiento debido a su naturaleza iterativa y menos eficiente.

¿Qué modelo usarían?
Para resolver problemas reales o trabajar con datasets más complejos, definitivamente usaría el modelo con librerías. Además de ser más rápido, logra mejores resultados de precisión y es más fácil de ajustar a través de la configuración de hiperparámetros como el kernel y la regularización. Aunque implementar un modelo desde cero es una excelente forma de entender el funcionamiento del SVM, no es práctico para aplicaciones en la vida real debido a sus limitaciones de rendimiento y escalabilidad.

